# 数据读取、清洗

In [1]:
#获取文件的编码格式函数
import chardet
def get_encoding(file):
    with open(file, 'rb') as f:
        return chardet.detect(f.read())['encoding']
encoding=get_encoding('G:\\PracticeData\\val.txt')
print(encoding)

GB2312


In [5]:
#该新闻文件只有200条新闻
import pandas as pd
import numpy
news = pd.read_table('G:\\PracticeData\\news_tensite_xml.smarty.txt',header = None,encoding = 'GB18030')
# 分别读取url,content,contenttitle内容
url = []
content = []
contenttitle = []
news.shape  #可判断出文件读取后的数据类型是矩阵类型，news是1200行1列的矩阵

(1200, 1)

In [6]:
# 遍历所有行第0列,获取相应的url、标题 、新闻内容加入对应的列表
for i in range(news.shape[0]):
    if '<contenttitle>' in news[0][i]:
        contenttitle.append(news[0][i])
    if '<url>' in news[0][i]:
        url.append(news[0][i])
    if '<content>' in news[0][i]:
        content.append(news[0][i])
news1= pd.DataFrame({'url':url,'contenttitle':contenttitle,'content':content})
print(news1.shape)
news1.tail(3)

(200, 3)


,url,contenttitle,content
197,<url>http://news.cn.yahoo.com/ypen/20120612/11...,<contenttitle>“走婚族”悄然兴起 六成网友称能接受</contenttitle>,<content>虽然结婚，却没有共同住所，平日各回各家，节假日才聚到一起——网友们把这样的...
198,<url>http://news.cn.yahoo.com/newspic/news/692...,<contenttitle>全国掀起扫黄风暴 涉黄场所天上落人间</contenttitle>,<content>中国雅虎侠客平台仅提供信息存储空间服务，其内容均由上网用户提供。中国雅虎提...
199,<url>http://news.cn.yahoo.com/ypen/20120613/11...,<contenttitle>缅甸若开邦骚乱局势总体得到控制</contenttitle>,<content>新华网仰光６月１２日电（记者张云飞）实行紧急状态的缅甸西部若开邦１２日虽然...


In [9]:
#去掉前后标签
new_contenttitle = []
new_content = []
new_url = []
for i in range(len(url)):
    new_contenttitle.append(contenttitle[i].split('>')[1].split('<')[0])
    new_url.append(url[i].split('>')[1].split('<')[0])
    new_content.append(content[i].split('>')[1].split('<')[0])
new_news = pd.DataFrame({'url':new_url,'contenttitle':new_contenttitle,'content':new_content})
new_news.tail(3)

,url,contenttitle,content
197,http://news.cn.yahoo.com/ypen/20120612/1106081...,“走婚族”悄然兴起 六成网友称能接受,虽然结婚，却没有共同住所，平日各回各家，节假日才聚到一起——网友们把这样的结婚者称为走婚族。...
198,http://news.cn.yahoo.com/newspic/news/6922/33/...,全国掀起扫黄风暴 涉黄场所天上落人间,中国雅虎侠客平台仅提供信息存储空间服务，其内容均由上网用户提供。中国雅虎提示您尊重权利人之权...
199,http://news.cn.yahoo.com/ypen/20120613/1107633...,缅甸若开邦骚乱局势总体得到控制,新华网仰光６月１２日电（记者张云飞）实行紧急状态的缅甸西部若开邦１２日虽然仍有零星骚乱发生，...


In [18]:
content=new_news.content.values.tolist()#转换为list
print(content[len(content)-1])  #输出list中最后一个新闻，第200条新闻

新华网仰光６月１２日电（记者张云飞）实行紧急状态的缅甸西部若开邦１２日虽然仍有零星骚乱发生，但局势总体已得到控制。据官方最新统计，自８日在若开邦孟都等地区实行宵禁至１１日，共有２１人在骚乱中死亡，２１人受伤，１６６２间房屋被烧。１２日上午，位于若开邦首府实兑的纳西市场遭纵火。此外，实兑附近农村也有几处房屋被烧。军警在努力恢复当地秩序的同时，还为临时救助站的难民提供救济物资。目前，在孟都和布帝洞地区共有８个临时救助站，其中有两处专门用来安置穆斯林难民。此次骚乱的直接原因是近日发生的两起杀人案，激化了不同民族和宗教之间的对立。缅甸中央政府已于６日成立专门调查委员会，并从８日开始在若开邦部分地区实行宵禁，白天则禁止５人以上在公共场合集会。１０日，缅甸总统吴登盛签署法令宣布在若开邦立即实行紧急状态，以防止骚乱事态进一步扩大蔓延。


# 分词

In [22]:
#对每一篇新闻进行分词，并将每一篇分好词的新闻放进DataFrame结构中
import jieba
content_word=[]
for line in content:
    current_segment=jieba.lcut(line)#对每一篇新闻进行分词
    if len(current_segment)>1 and current_segment!='\r\n':
        content_word.append(current_segment)
#content_word[len(content_word)-1]#输出最后一篇新闻的分词情况
df_content=pd.DataFrame({'content_word':content_word})#构造dataframe数据
df_content.tail(3)#分词之后只有166条新闻，通过if条件语句删除了34条无效新闻

,content_word
163,"[虽然, 结婚, ，, 却, 没有, 共同, 住所, ，, 平日, 各回, 各家, ，, 节..."
164,"[中国, 雅虎, 侠客, 平台, 仅, 提供, 信息, 存储空间, 服务, ，, 其, 内容..."
165,"[新华网, 仰光, ６, 月, １, ２, 日电, （, 记者, 张云飞, ）, 实行, 紧..."


# 去除停用词

In [54]:
#读取停止词，可根据需求适量增加，同样使用panda展示出来 ，这里取样为前20个
stopwords=pd.read_table('G:/PracticeData/stopwords.txt',index_col=False,sep='\n',quoting=3,encoding='GB18030',names=['stopwords'])
#stopwords.head(20)

#定义一个去除停用词的函数
def drop_stopwords(df_content,stopwords):
    content_clean=[]
    all_words=[]
    for line in df_content:
        line_clean=[]
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word))
        content_clean.append(line_clean)
    return content_clean,all_words

contents =df_content.content_word.values.tolist()  #分好词的新闻内容的列表（词列表）
stopwords=stopwords.stopwords.values.tolist()      #停用词的列表
contents_clean,all_words=drop_stopwords(contents, stopwords)   #将上方的两个列表作为输入参数，输入定义的函数中

df_content2=pd.DataFrame({'content_clean':contents_clean})
print(df_content2.tail())

#drop停止词后，可视化显示所有的词，然后根据这些词进行分组统计，查看每一词在文章中出现的次数
df_all_words=pd.DataFrame({'all_words':all_words})
df_all_words.head(3)

                                         content_clean
161  [一位, 做, 足, 防晒, 措施, 光头, 市民, 头上, 挂满, 汗珠, 记者, 黄澄锋...
162  [中新网, 月, 日电, 俄新网, 报道, 美国国务院, 发言人, 纽兰, 美国, 俄罗斯,...
163  [结婚, 住所, 平日, 各回, 各家, 节假日, 聚到, 网友, 结婚, 称为, 走, 婚...
164  [中国, 雅虎, 侠客, 平台, 提供, 信息, 存储空间, 服务, 内容, 上网, 用户,...
165  [新华网, 仰光, 月, 日电, 记者, 张云飞, 实行, 紧急状态, 缅甸, 西部, 开邦...


,all_words
0,中广网
1,唐山
2,月


In [55]:
words_count=df_all_words.groupby(by=['all_words'])['all_words'].agg({'count': numpy.size})
words_count=words_count.reset_index().sort_values( by=['count'],ascending=False)
words_count.head(10)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


,all_words,count
8345,,833
4680,月,254
4513,日,209
3363,年,152
449,中国,146
7842,银行,132
7065,记者,131
3968,投资,126
435,中,120
7152,说,111


In [44]:
#调用jieba.analyse.extract_tags函数，提取出关键词
from jieba.analyse import *
data=new_news['content'][0]
#使用TF-idf（词频-逆文档频率）方式提取关键词和权重
for keyword,weight in extract_tags(data,topK=5,withWeight=True):
    print(keyword,weight)
print()
for keyword,weight in textrank(new_news['content'][150],topK=5,withWeight=True):
    print(keyword,weight)

枪支 0.2766975807191223
犯罪 0.2107448868282181
销毁 0.16950587698047873
非法 0.11831675845260638
公安部 0.11695122577234042

台湾 1.0
惊传 0.8414535319341819
身亡 0.8226442564879378
手续 0.7918254786577176
准备 0.7858783141581096


In [50]:
import jieba.analyse
#print(df_content['content_word'][50])
df_content_str="".join(df_content['content_word'][165])
print(df_content_str)
for keyword in jieba.analyse.extract_tags(df_content_str,topK=5):
    print(keyword)

新华网仰光６月１２日电（记者张云飞）实行紧急状态的缅甸西部若开邦１２日虽然仍有零星骚乱发生，但局势总体已得到控制。据官方最新统计，自８日在若开邦孟都等地区实行宵禁至１１日，共有２１人在骚乱中死亡，２１人受伤，１６６２间房屋被烧。１２日上午，位于若开邦首府实兑的纳西市场遭纵火。此外，实兑附近农村也有几处房屋被烧。军警在努力恢复当地秩序的同时，还为临时救助站的难民提供救济物资。目前，在孟都和布帝洞地区共有８个临时救助站，其中有两处专门用来安置穆斯林难民。此次骚乱的直接原因是近日发生的两起杀人案，激化了不同民族和宗教之间的对立。缅甸中央政府已于６日成立专门调查委员会，并从８日开始在若开邦部分地区实行宵禁，白天则禁止５人以上在公共场合集会。１０日，缅甸总统吴登盛签署法令宣布在若开邦立即实行紧急状态，以防止骚乱事态进一步扩大蔓延。
开邦
骚乱
救助站
实兑
缅甸
